In [1]:
import graphlab as gl
import time
import json
import re

A newer version of GraphLab Create (v1.9) is available! Your current version is v1.7.1.

You can use pip to upgrade the graphlab-create package. For more information see https://dato.com/products/create/upgrade.


In [45]:
timingsList = ["12:30 PM to 3:30 PM, 7 PM to 11 PM", "11 AM to 11 PM", "10:30 AM to 11 PM", "12 Noon to 11 PM (Mon-Thu), 12 Noon to 1 AM (Fri-Sun)",
              "9 AM to 11 PM", "12:30 PM to 3:30 PM, Tue-Thu, Sunday Closed", "11 AM to 11:30 PM", "12:30 PM to 11 PM", "12 Noon to 3 PM, 7 PM to 10:15 PM",
              "12 Noon to 11 PM (Tue-Thu, Sat-Sun), 1:30 PM to 11 PM (Fri), Mon Closed",
                "9 AM to 11 PM, 12 Noon to 11 PM (Tue-Wed, Thurs-Fri), 1:30 PM to 11 PM (Sat), Sun-Mon Closed",
              "9 AM to 11 PM, 12 Noon to 11 PM (Tue-Wed, Thurs-Fri), 1:30 PM to 11 PM (Sat), Sunday, Mon Closed"]

In [32]:
def hoursAndMinsCalculate(startTime):
    if(':' in startTime):
#        print startTime
        hoursAndMins = re.search("""(.*?):(.*)""",startTime, re.I|re.S)
#        print "ya", hoursAndMins.group(1), type(hoursAndMins.group(1)), len(hoursAndMins.group(1))
        hours = hoursAndMins.group(1)
        mins = hoursAndMins.group(2)
        timings = str(int(hours)*100 + int(mins))
    else:
        timings = str(int(startTime)*100)
#    print "yup", timings
    return timings

In [41]:
def timingsCalculate(eachTime):
    timings = ""
    startTime = re.search("""(.*?)\sto""",eachTime,re.I|re.S).group(1)
    if("AM" in startTime):
        startTime = re.search("""(.*?)\sAM""",startTime, re.I|re.S).group(1)
        timings = hoursAndMinsCalculate(startTime) + "-"
    elif("PM" in startTime):
        startTime = re.search("""(.*?)\sPM""",startTime, re.I|re.S).group(1)
        tom = hoursAndMinsCalculate(startTime) 
        if( "12" not in tom ):
            timings = str(int(tom) + 1200) + "-"
        else:
            timings = tom + "-"
    elif("on" in startTime):
        timings = "1200-"

    endTime = re.search("""to\s(.*)""",eachTime,re.I|re.S).group(1)
    if("AM" in endTime):
        endTime = re.search("""(.*?)\sAM""",endTime, re.I|re.S).group(1)
        if (int(hoursAndMinsCalculate(endTime)) < 300):
            timings = timings + str(int(hoursAndMinsCalculate(endTime)) + 2400)
        else:
            timings = timings + hoursAndMinsCalculate(endTime)
    elif("PM" in endTime):
        endTime = re.search("""(.*?)\sPM""",endTime, re.I|re.S).group(1)
        timings = timings + str(int(hoursAndMinsCalculate(endTime)) + 1200)

    elif("on" in endTime):
        timings = timings + "1200"

    return timings

In [34]:
#Monday, Tuesday, Wednesday, Thursday, Friday, Saturday, Sunday
def checkClosedDay(whichDay):
    if("on" in whichDay):
        return 0
    elif("ue" in whichDay):
        return 1
    elif("ed" in whichDay):
        return 2
    elif("hu" in whichDay):
        return 3
    elif("ri" in whichDay):
        return 4
    elif("at" in whichDay):
        return 5
    else:
        return 6

In [29]:
#last occurence of the character ')' everything after this would be the days for which the restaurant is closed.
def setClosedDays(eachString):
    ch = re.search("""\),(.*?)Closed""", eachString, re.I|re.S)
    if(ch == None):
        eachString = eachString.split('to')
        ch = re.search("""M,(.*?)Closed""", eachString[len(eachString)-1], re.I|re.S)

    daysClosed = re.sub("""\s""", "", ch.group(1), re.I|re.S)
    daysClosed = daysClosed.split(',')
    listOfTime = ["","","","","","",""]
    for eachTerm in daysClosed:
        if('-' in eachTerm):
            checkingDays = eachTerm.split('-')
            num = checkClosedDay(checkingDays[0])
            num1 = checkClosedDay(checkingDays[1])
            if(num < num1):
                while(num <= num1):
                    listOfTime[num] = "No"
                    num = num + 1
            else:
                while(num <= 6):
                    listOfTime[num] = "No"
                    num = num + 1
                num = 0
                while(num <= num1):
                    listOfTime[num] = "No"
                    num = num + 1

        else:
            num = checkClosedDay(eachTerm)
            listOfTime[num] = "No"
    return listOfTime

In [46]:
#timingsList = ["12:30 PM to 3 PM (Friday), Monday Closed"]
for eachTime in timingsList: 
    print eachTime
    listOfDaysTime = ["","","","","","",""]
    timings = ""    
    #mon - 0, tue - 1, wed - 2, thurs - 3, fri -4, sat - 5
    if("Closed" in eachTime):
        listOfDaysTime = setClosedDays(eachTime)

    s = re.search("""(.*)\),(.*?)Closed""", eachTime, re.I|re.S)
    if(s != None):
        s = s.group(1)
        s = s + ")"
    else:
        s = eachTime

    s1 = re.findall("""(.*?)\)""", s, re.I|re.S)
    if(len(s1) != 0):
        for k in range(len(s1)):
            if(s1[k][:2] == ", "):
                s1[k] = s1[k][2:]

        for e in s1:
            searchBracketTime = re.search("""(.*?)\(""", e, re.I|re.S).group(1)
            searchBracketDays = re.search("""\((.*)""", e, re.I|re.S).group(1)
            searchBracketDays = searchBracketDays.split(',')
            detectComma = re.findall("""to""",searchBracketTime,re.I|re.S) #more than one time present
            numTimes = len(detectComma)
            if(numTimes == 1):
                timings = timingsCalculate(searchBracketTime)
            if(numTimes > 1):
                listOfTimings = searchBracketTime.split(',')
                for p in range(len(listOfTimings)):
                    if(listOfTimings[p][len(listOfTimings[p])-1] == ' '):
                        listOfTimings[p] = listOfTimings[p][:len(listOfTimings[p])-1]

                for m in range(len(listOfTimings)):
                    timings = timings + timingsCalculate(listOfTimings[m]) + '#'

            for eachTerm in searchBracketDays:
                if('-' in eachTerm):
                    checkingDays = eachTerm.split('-')
                    num = checkClosedDay(checkingDays[0])
                    num1 = checkClosedDay(checkingDays[1])
                    if(num < num1):
                        while(num <= num1):
                            listOfDaysTime[num] = timings
                            num = num + 1
                    else:
                        while(num <= 6):
                            listOfDaysTime[num] = timings
                            num = num + 1
                        num = 0
                        while(num <= num1):
                            listOfDaysTime[num] = timings
                            num = num + 1

                else:
                    num = checkClosedDay(eachTerm)
                    listOfDaysTime[num] = timings            

    else:
        detectComma = re.findall("""to""",eachTime,re.I|re.S) #more than one time present
        numTimes = len(detectComma)
        timings = ""
        if(numTimes == 1):
            timings = timingsCalculate(eachTime)
        if(numTimes > 1):
            listOfTimings = eachTime.split(',')
            for m in range(len(listOfTimings)):
                timings = timings + timingsCalculate(listOfTimings[m]) + '#'

    for t in range(len(listOfDaysTime)):
        if(listOfDaysTime[t] == ""):
            listOfDaysTime[t] = timings

                
    print listOfDaysTime

12:30 PM to 3:30 PM, 7 PM to 11 PM
['1230-1530#1900-2300#', '1230-1530#1900-2300#', '1230-1530#1900-2300#', '1230-1530#1900-2300#', '1230-1530#1900-2300#', '1230-1530#1900-2300#', '1230-1530#1900-2300#']
11 AM to 11 PM
['1100-2300', '1100-2300', '1100-2300', '1100-2300', '1100-2300', '1100-2300', '1100-2300']
10:30 AM to 11 PM
['1030-2300', '1030-2300', '1030-2300', '1030-2300', '1030-2300', '1030-2300', '1030-2300']
12 Noon to 11 PM (Mon-Thu), 12 Noon to 1 AM (Fri-Sun)
['1200-2300', '1200-2300', '1200-2300', '1200-2300', '1200-2500', '1200-2500', '1200-2500']
9 AM to 11 PM
['900-2300', '900-2300', '900-2300', '900-2300', '900-2300', '900-2300', '900-2300']
12:30 PM to 3:30 PM, Tue-Thu, Sunday Closed
12:30 PM to 3:30 PM, Tue-Thu, Sunday Closed
['1230-1530', 'No', 'No', 'No', '1230-1530', '1230-1530', 'No']
11 AM to 11:30 PM
['1100-2330', '1100-2330', '1100-2330', '1100-2330', '1100-2330', '1100-2330', '1100-2330']
12:30 PM to 11 PM
['1230-2300', '1230-2300', '1230-2300', '1230-2300', '

In [26]:
ss = "9 AM to 11 PM, 12:30 PM to 3:30 PM (Tue-Thu, Sat-Sun), 12:30 PM to 3 PM (Friday), Monday Closed"
s = re.search("""(.*)\),(.*?)Closed""", ss, re.I|re.S).group(1)
s = s + ")"
#print s
s1 = re.findall("""(.*?)\)""", s, re.I|re.S)
for k in range(len(s1)):
    if(s1[k][:2] == ", "):
        s1[k] = s1[k][2:]
    #s1[k] = s1[k] + ")"
for e in s1:
    searchBracketTime = re.search("""(.*?)\(""", e, re.I|re.S).group(1)
    searchBracketDays = re.search("""\((.*)""", e, re.I|re.S).group(1)
    searchBracketDays = searchBracketDays.split(',')
    listOfTime1 = ["","","","","","",""]
    for eachTerm in searchBracketDays:
        if('-' in eachTerm):
            checkingDays = eachTerm.split('-')
            num = checkClosedDay(checkingDays[0])
            num1 = checkClosedDay(checkingDays[1])
            if(num < num1):
                while(num <= num1):
                    listOfTime1[num] = "No"
                    num = num + 1
            else:
                while(num <= 6):
                    listOfTime1[num] = "No"
                    num = num + 1
                num = 0
                while(num <= num1):
                    listOfTime1[num] = "No"
                    num = num + 1

        else:
            num = checkClosedDay(eachTerm)
            listOfTime1[num] = "No"
    return listOfTime1

9 AM to 11 PM, 12:30 PM to 3:30 PM 
Tue-Thu, Sat-Sun
12:30 PM to 3 PM 
Friday


In [ ]:
#CheckPoint - old code !!! 
for eachTime in timingsList2 : 
    #print eachTime
    detectComma = re.findall("""to""",eachTime,re.I|re.S) #more than one time present
    numTimes = len(detectComma)
    timings = ""
    #should closed be checked here!
    if(numTimes == 1):
        timings = timingsCalculate(eachTime)
    if((numTimes == 2) and ("(" not in eachTime)):
        listOfTimings = eachTime.split(',')
        timings = timingsCalculate(listOfTimings[0]) + '#'
        timings = timings + timingsCalculate(listOfTimings[1])
    
    if((numTimes == 3) and ("(" not in eachTime)):
        listOfTimings = eachTime.split(',')
        timings = timingsCalculate(listOfTimings[0]) + '#'
        timings = timings + timingsCalculate(listOfTimings[1]) + '#'
        timings = timings + timingsCalculate(listOfTimings[2]) 
    
    #mon - 0, tue - 1, wed - 2, thurs - 3, fri -4, sat - 5
    if("Closed" in eachTime):
        
        ti = re.search("""(.*?)\s\(""",eachTime,re.I|re.S).group(1)
        detectComma = re.findall("""to""",ti,re.I|re.S) #more than one time present
        numTimes = len(detectComma)
        #11 AM to 11 PM (Mon-Sat), Sun Closed
        if(numTimes == 1):
            timings = timingsCalculate(ti)
        if(numTimes == 2):
            listOfTimings = ti.split(',')
            timings = timingsCalculate(listOfTimings[0]) + '#'
            timings = timings + timingsCalculate(listOfTimings[1])
    
        whichDay = re.search("""(.*?)\sClosed""", eachTime, re.I|re.S).group(1)
        print "i am here", whichDay
        #12:30 PM to 3:30 PM (Mon-Sat), Sun Closed 
        if(whichDay[-4] == " "):#if abbreviations are used! What about Thursday check for Mon,Tue,Wed,Thurs,Fri,Sat,Sun
            if(whichDay[-6] == ")"):
                whichDayClosed = checkClosedDay(whichDay)
                print whichDayClosed
            else:
                #multiples days closed
                asd = re.search("""\\),(.*?)""", whichDay, re.I|re.S).group(1)
                print asd, "asdsadasd"
        else:#
            print checkClosedDay(whichDay)

In [28]:
a = "12:30 PM to 3:30 PM, Tue-Thu, Sunday Closed"
a = a.split('to')
s = re.search("""M,(.*?)Closed""", a[len(a)-1], re.I|re.S)
if(s != None):
    print "yes", s.group(1)

yes  Tue-Thu, Sunday 


In [ ]:
#for days name when fullNames are given! The starting letter's case does not matter since not checking for that!
def checkClosedDay1(whichDay):
    if((whichDay[-5:-3]) == "on"):
        return 1
    elif((whichDay[-6:-3]) == "ues"):
        return 2
    elif((whichDay[-6:-3]) == "nes"):
        return 3
    elif((whichDay[-6:-3]) == "urs"):
        return 4
    elif((whichDay[-5:-3]) == "ri"):
        return 5
    elif((whichDay[-5:-3]) == "ur"):
        return 6
    else:
        return 7